In [3]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import torch
from transformers import pipeline
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import openpyxl
import re
from tqdm import tqdm_notebook as tqdm

In [4]:
# importing the final df of all parties.
df_final_tweets = pd.read_csv('../data/processed/df_final_2000.csv')
df_final_mentions = pd.read_csv('../data/processed/df_final_mentions_2000.csv')

In [5]:
# creating just one dataframe
frames = [df_final_tweets, df_final_mentions]
df_total = pd.concat(frames, ignore_index=True)

In [6]:
df_total.drop(labels='Unnamed: 0', axis=1)

,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count,partido,tipo de post,hashtags
0,Partido Popular,2021-04-24 09:46:56+00:00,1385893007694700553,➡️ A partir de las 12:00h. @pablocasado_ e @Id...,"[0, 227]",['Twitter Web App'],18,38,821483,5291,102573,Partido Popular,publicación,"['YoConAyuso', 'VotaLIBERTAD']"
1,Partido Popular,2021-04-24 09:16:38+00:00,1385885381996986370,De los más de 50M de € que Sánchez otorgó a un...,"[0, 268]",['Twitter Web App'],57,102,821483,5291,102573,Partido Popular,publicación,[]
2,Partido Popular,2021-04-24 07:56:46+00:00,1385865284309987329,El BOE no es un instrumento que Sánchez pueda ...,"[0, 279]",['Twitter for iPhone'],101,202,821483,5291,102573,Partido Popular,publicación,[]
3,Partido Popular,2021-04-24 06:01:06+00:00,1385836175345262592,"☀ Buenos días y #FelizSábado a todos, aquí os...","[0, 119]",['Twitter for iPhone'],11,28,821483,5291,102573,Partido Popular,publicación,['FelizSábado']
4,Partido Popular,2021-04-23 20:10:16+00:00,1385687486865760256,“Somos libres porque vivimos en Madrid”.\n\n👉 ...,"[0, 55]",['Twitter for iPhone'],228,737,821483,5291,102573,Partido Popular,publicación,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16172,Diario ¶ El Cambio,2021-04-24 10:07:13+00:00,1385898114847068160,Vox es una amenaza para la democracia https://...,"[0, 61]",['Twitter for iPad'],0,0,27,127,7612,Vox,mención,[]
16173,Todo Por Hacer,2021-04-24 10:07:13+00:00,1385898113890758656,Cuando se hace necesario tender una línea roja...,"[0, 278]",['Twitter Web App'],1,5,13323,969,15280,Vox,mención,[]
16174,Pedro G. 🇪🇸,2021-04-24 10:07:13+00:00,1385898111659356162,@Eligorricho @Santi_ABASCAL @Macarena_Olona @O...,"[176, 413]",['Twitter for iPhone'],0,1,824,842,664,Vox,mención,[]
16175,Lolo 🇪🇸,2021-04-24 10:07:12+00:00,1385898108245118976,"@GuajeSalvaje Que vox busca que? No, valles ca...","[14, 72]",['Twitter for Android'],0,0,21,34,3110,Vox,mención,[]


## Machine learning model: BERT

This model classify text into 1, 2, 3, 4 or 5 starts. Being 1 the most negative and 5 the most positive. 

In [7]:
def sentiment_classificaton(column):
    
    model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)
    
    list_tweets = column.tolist()
    
    results = classifier(list_tweets[:8000])
    
    sentiment_list = []
    scores = []

    for result in results:
        scores_list = result['score']
        scores.append(scores_list)
    
        for i in range(len(result)):
            results_list = result['label'][i]
            if results_list != " ":
                sentiment_list.append(results_list)
                
    return sentiment_list, scores

In [8]:
sentiment_classificaton(df_total['full_text'])

KeyboardInterrupt: 

In [ ]:
df_total['sentimiento'] = sentiment_list
df_total['scores'] = scores

## Exporting the final data frame 

In [138]:
df_total

,Unnamed: 0,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count,entities.hashtags,entities.urls,partido,tipo de post,sentimiento,scores
0,0,Partido Popular,2021-04-17 10:29:44+00:00,1383367063666782213,👉 Javier Fernández-Lasquetty: “Todos los socia...,"[0, 252]",['Twitter Media Studio - LiveCut'],2,2,820639,5291,102263,"[{'text': 'YoConAyuso', 'indices': [241, 252]}]",[],Partido Popular,publicación,1,0.623746
1,1,Partido Popular,2021-04-17 10:25:21+00:00,1383365962888409090,👉 Javier Fernández-Lasquetty: “Libertad para t...,"[0, 163]",['Twitter Media Studio - LiveCut'],7,11,820639,5291,102263,"[{'text': 'YoConAyuso', 'indices': [152, 163]}]",[],Partido Popular,publicación,5,0.331548
2,2,Partido Popular,2021-04-17 10:16:32+00:00,1383363740356145161,📡 #EnDirecto👉 @IdiazAyuso atiende a los medios...,"[0, 103]",['Twitter Media Studio'],16,34,820639,5291,102263,"[{'text': 'EnDirecto', 'indices': [2, 12]}]","[{'url': 'https://t.co/N4H746boKu', 'expanded_...",Partido Popular,publicación,5,0.310655
3,3,Partido Popular,2021-04-17 09:59:36+00:00,1383359481254268936,"🔵 En unos minutos, no te pierdas la intervenci...","[0, 179]",['Twitter Web App'],37,49,820639,5291,102263,"[{'text': 'YoConAyuso', 'indices': [168, 179]}]","[{'url': 'https://t.co/6WGpUW8ul9', 'expanded_...",Partido Popular,publicación,5,0.409290
4,4,Partido Popular,2021-04-17 06:14:00+00:00,1383302705779671044,☕️ Muy buenos días y #FelizSábado a todos 😊\n\...,"[0, 102]",['Twitter Web App'],8,26,820639,5291,102263,"[{'text': 'FelizSábado', 'indices': [21, 33]}]","[{'url': 'https://t.co/3quq6iOQY2', 'expanded_...",Partido Popular,publicación,5,0.695196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,95,Vox_Hellin,2021-04-17 10:58:55+00:00,1383374409545576455,‼️Nosotros luchamos por todos los españoles si...,"[0, 158]",['Twitter for Android'],0,0,290,338,3340,"[{'text': 'ProtegeMadrid', 'indices': [125, 13...",[],Vox,mención,5,0.597425
996,96,Francisco J.rey /🌹❤,2021-04-17 10:58:49+00:00,1383374384346267651,@Sirona33014070 @TaniaCrespo3 @LorenaCastro20 ...,"[60, 198]",['Twitter for Android'],0,0,1005,991,5732,[],[],Vox,mención,1,0.557505
997,97,christian9516,2021-04-17 10:58:40+00:00,1383374347344125957,@TabarniaB Sería noticia que Vox no llenará el...,"[11, 63]",['Twitter for Android'],0,0,23,75,2859,[],[],Vox,mención,1,0.492922
998,98,justi,2021-04-17 10:58:21+00:00,1383374264561061891,El doble pinchazo de Vox en Orcasitas: ni públ...,"[0, 87]",['Twitter for iPhone'],0,0,954,976,81113,[],"[{'url': 'https://t.co/aNGIIqo431', 'expanded_...",Vox,mención,1,0.677282


In [137]:
df_total.to_excel('../data/processed/df_total_SENTIMENT_2.xlsx')

NameError: name 'dfw' is not defined